In [72]:
from nowcasting_dataset.datamodule import NowcastingDataModule
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import numcodecs
import gcsfs
from typing import List
import io
import hashlib
import os
import glob

import logging
logging.basicConfig()
logger = logging.getLogger('nowcasting_dataset')
logger.setLevel(logging.DEBUG)

## Load Zarr batches

In [63]:
SRC_ZARR_FILENAME = 'gs://solar-pv-nowcasting-data/prepared_ML_training_data/testing.zarr'
DST_NETCDF4_PATH = 'gs://solar-pv-nowcasting-data/prepared_ML_training_data/netcdf4/'
LOCAL_TEMP_PATH = Path('~/temp/').expanduser()

In [64]:
DST_NETCDF4_PATH

'gs://solar-pv-nowcasting-data/prepared_ML_training_data/netcdf4/'

In [3]:
%%time
ds = xr.open_dataset(SRC_ZARR_FILENAME, engine='zarr', chunks={})

CPU times: user 21.8 s, sys: 935 ms, total: 22.7 s
Wall time: 35.5 s


In [4]:
ds

<xarray.Dataset>
Dimensions:               (example: 974880, nwp_variable: 10, nwp_x: 2, nwp_y: 2, sat_variable: 12, sat_x: 32, sat_y: 32, time: 19)
Coordinates:
  * example               (example) int32 0 1 2 3 ... 974877 974878 974879
    init_time             (example) datetime64[ns] dask.array<chunksize=(5442,), meta=np.ndarray>
  * nwp_variable          (nwp_variable) <U5 't' 'dswrf' 'prate' ... 'mcc' 'hcc'
  * nwp_x                 (nwp_x) int32 0 1
  * nwp_y                 (nwp_y) int32 0 1
  * sat_variable          (sat_variable) <U6 'HRV' 'IR_016' ... 'WV_073'
  * sat_x                 (sat_x) int32 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * sat_y                 (sat_y) int32 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * time                  (time) int32 0 1 2 3 4 5 6 7 ... 12 13 14 15 16 17 18
Data variables:
    nwp                   (example, nwp_variable, time, nwp_y, nwp_x) float32 dask.array<chunksize=(32, 10, 19, 2, 2), meta=np.ndarray>
    nwp_time_coords       (example, time) datetime64[ns] dask.array<chunksize=(32, 19), meta=np.ndarray>
    nwp_x_coords          (example, nwp_x) float32 dask.array<chunksize=(32, 2), meta=np.ndarray>
    nwp_y_coords          (example, nwp_y) float32 dask.array<chunksize=(32, 2), meta=np.ndarray>
    pv_system_id          (example) int32 dask.array<chunksize=(32,), meta=np.ndarray>
    pv_system_row_number  (example) int32 dask.array<chunksize=(32,), meta=np.ndarray>
    pv_yield              (example, time) float32 dask.array<chunksize=(32, 19), meta=np.ndarray>
    pv_yield_time_coords  (example, time) datetime64[ns] dask.array<chunksize=(32, 19), meta=np.ndarray>
    sat_data              (example, time, sat_y, sat_x, sat_variable) float32 dask.array<chunksize=(32, 19, 32, 32, 12), meta=np.ndarray>
    sat_time_coords       (example, time) datetime64[ns] dask.array<chunksize=(32, 19), meta=np.ndarray>
    sat_x_coords          (example, sat_x) int32 dask.array<chunksize=(32, 32), meta=np.ndarray>
    sat_y_coords          (example, sat_y) int32 dask.array<chunksize=(32, 32), meta=np.ndarray>

In [52]:
batch_size = 32
n_batches = len(ds.example) // batch_size

gcs = gcsfs.GCSFileSystem()

In [81]:
def get_filename(batch_i: int) -> Path:
    """Generate full filename, excluding path.
    
    Filename includes the first 6 digits of the MD5 hash of the filename,
    as recommended by Google Cloud in order to distribute data across
    multiple back-end servers.
    """
    filename = f'{batch_i}.nc'
    hash_of_filename = hashlib.md5(filename.encode()).hexdigest()
    return f'{hash_of_filename[:6]}_{filename}'


def write_batch_locally(batch: xr.Dataset, batch_i: int):
    encoding = {
        name: {'compression': 'lzf'}
        for name in batch.data_vars}
    filename = get_filename(batch_i)
    local_filename = LOCAL_TEMP_PATH / filename
    batch.to_netcdf(local_filename, engine='h5netcdf', mode='w', encoding=encoding)

In [82]:
DST_NETCDF4_PATH

'gs://solar-pv-nowcasting-data/prepared_ML_training_data/netcdf4/'

In [83]:
UPLOAD_EVERY_N_BATCHES = 64

for batch_i in range(n_batches):
    print(f'\r{batch_i}/{n_batches}', end='', flush=True)
    start_example = batch_i * batch_size
    end_example = start_example + batch_size - 1  # -1 because slice gets range [start, end] not [start, end).
    batch = ds.sel(example=slice(start_example, end_example)).load()
    write_batch_locally(batch, batch_i)
    if batch_i > 0 and (batch_i % UPLOAD_EVERY_N_BATCHES) == 0:
        print('\ruploading***************', end='', flush=True)
        gcs.put(str(LOCAL_TEMP_PATH), DST_NETCDF4_PATH, recursive=True)
        files = glob.glob(str(LOCAL_TEMP_PATH / '*.nc'))
        for f in files:
            os.remove(f)
    
print()
print('Done!')

uploading***************
117/30465

/home/jack/miniconda3/envs/nowcasting_dataset/lib/python3.9/concurrent/futures/thread.py:167: RuntimeWarning: coroutine 'GCSFileSystem._put_file' was never awaited
  w = _WorkItem(f, fn, args, kwargs)
/home/jack/miniconda3/envs/nowcasting_dataset/lib/python3.9/concurrent/futures/thread.py:167: RuntimeWarning: coroutine 'GCSFileSystem._put_file' was never awaited
  w = _WorkItem(f, fn, args, kwargs)


uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
uploading***************
